In [ ]:
!pip install keras_self_attention
from keras_self_attention import SeqSelfAttention
!pip install bangla-stemmer
!pip install bnltk
!pip install bnlp_toolkit
!pip install tweet-preprocessor
!pip install wordcloud
!pip install https://github.com/JonathanRaiman/glove/archive/master.zip
from bangla_stemmer.stemmer import stemmer
from bnltk.stemmer import BanglaStemmer
from bnltk.tokenize import Tokenizers
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import re
%matplotlib inline

#!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import PorterStemmer


from wordcloud import WordCloud


import preprocessor as p
import keras
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Conv1D,GRU,Concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model

from gensim.models import KeyedVectors

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
from bnltk.tokenize import Tokenizers
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.preprocessing import LabelEncoder
import gensim
from gensim.test.utils import datapath
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
MAX_NUM_WORDS =26000
MAX_SEQ_LENGTH = 140
EMBEDDING_DIM = 300
newdf = []
def clean_tweets(tweets,stem=True, stopword=True):
    stopwordsBN = set(line.strip() for line in open('../input/bn-stopwords/stopwords.txt'))
    stmr = stemmer.BanglaStemmer()
    bn_stemmer = BanglaStemmer()
    cleaned_tweets = []
    a = ""
    tweet = tweets
    word_tokens = nltk.word_tokenize(tweet)
    for w in range(len(word_tokens)):
        a = word_tokens[w]            
        a = "".join(i for i in a if  2432 <= ord(i) <= 2533 or ord(i)== 32)
        tweet = re.sub(' +', ' ', a)
        word_tokens[w] = tweet
    filtered_sentence = None
    if stem==True and stopwords==True:
        filtered_sentence = [bn_stemmer.stem(w) for w in word_tokens if not w in stopwordsBN]
    elif stem==True and stopword==False:
        filtered_sentence = [bn_stemmer.stem(w) for w in word_tokens]
    elif stem == False and stopword==True:
        filtered_sentence = [w for w in word_tokens if not w in stopwordsBN]
    else:
        filtered_sentence = [w for w in word_tokens]
    tweet = ' '.join(filtered_sentence)        
    tweet = str(tweet)
    word_tokens = nltk.word_tokenize(tweet)
    newdf.append(word_tokens)
    return tweet


def preprocess(corpus, stem = True, stopword=True):
    newdf = []
    ls = []
    idd=[]
    siz = []
    dicti = {}
    words=[]

    for i in corpus:
        j = clean_tweets(i,stem,stopword)
        ls.append(j)
    return ls


def convertToTensor(texts):
    print("Convert to tensor started")
    X = texts
    tokenizer= Tokenizer(num_words=MAX_NUM_WORDS)
    tokenizer.fit_on_texts(X)
    word_vector = tokenizer.texts_to_sequences(X)
    word_index = tokenizer.word_index
    vocab_size = len(word_index)
    dictionary = word_index
    np.save('dictionary.npy', dictionary) 
    t = Tokenizer(num_words=MAX_NUM_WORDS)
    read_dictionary = np.load('./dictionary.npy',allow_pickle='TRUE').item()
    t.word_index = read_dictionary
    #t.fit_on_texts(X)
    word_vector = t.texts_to_sequences(X)
    MAX_SEQ_LENGTH = 140
    input_tensor = pad_sequences(word_vector, maxlen=MAX_SEQ_LENGTH)
    print("Convert to tensor ended")
    return input_tensor,word_index

def Dataset_prepare(dataset=1):
    print("tokenizer")
    #dataset == 1 for BEmoC
    t = Tokenizers()
    df = None
    corpus = None
    y = None
    if dataset==1:
        df=pd.read_excel("/kaggle/input/emotiondataset/bemoc.xlsx")
        corpus = df['TEXT'].to_list()
        y=df['classes']
        return corpus,y
    elif dataset==3:
        df=pd.read_csv("/kaggle/input/emotionmerged/merged.csv")
        corpus = df['text'].to_list()
        y=df['class']
        
    else:
        df=pd.read_csv("/kaggle/input/banglaemotion/train.csv")
        corpus = df['text'].to_list()
        y=df['class']
        df2 = pd.read_csv("/kaggle/input/banglaemotion/test.csv")
        corpus2 = df2['text'].to_list()
        y2=df2['class']
        corpus.extend(corpus2)
        y = pd.concat([y,y2])
    print("end tokenizer")
    return corpus,y


def embedding_matrix_gen(word_index, embedding="fasttext"):
    print("embedding started")
    if str.lower(embedding)=="word2vec":
        model = Word2Vec.load("../input/bnword2vec/bengali_word2vec.model")
        model.build_vocab(newdf, update=True)
        model.train(newdf, total_examples=model.corpus_count, epochs=model.epochs)
        not_found = 0
        found = 0
        EMBEDDING_DIM =300
        embedding_matrix = np.zeros((MAX_NUM_WORDS, EMBEDDING_DIM))
        for (word, idx) in word_index.items():
            #print(idx)
            try:
                embedding_matrix[idx] =model.wv[word]
                found=found+1
            except:
                embedding_matrix[idx] =embedding_matrix[0]
                not_found = not_found+1
                continue
        print("embedding ended")
        return embedding_matrix,EMBEDDING_DIM
    elif str.lower(embedding)=="fasttext":
        print(len(newdf),newdf[0])
        model = gensim.models.FastText.load_fasttext_format('../input/fasttext-and-glove/bengali_fasttext_wiki.bin')
        #model.build_vocab(newdf, update=True)
        #model.train(newdf, total_examples=model.corpus_count, epochs=model.epochs)
        not_found = 0
        found = 0
        EMBEDDING_DIM =100
        embedding_matrix = np.zeros((MAX_NUM_WORDS, EMBEDDING_DIM))
        for (word, idx) in word_index.items():
            #print(idx)
            try:
                embedding_matrix[idx] =model.wv[word]
                found=found+1
            except:
                not_found = not_found+1
                continue
        print("embedding ended")
        return embedding_matrix,EMBEDDING_DIM 

    elif str.lower(embedding)=="glove":
        glove_file = '../input/fasttext-and-glove/bn_glove.39M.300d.txt'
        tmp_file = get_tmpfile("test_word2vec.txt")
        _ = glove2word2vec(glove_file, tmp_file)
        model = KeyedVectors.load_word2vec_format(tmp_file)
        EMBEDDING_DIM = 300
        embedding_matrix = np.zeros((MAX_NUM_WORDS, EMBEDDING_DIM))
        for (word, idx) in word_index.items():
            #print(idx)
            try:
                embedding_matrix[idx] =model[word]
            except:
                embedding_matrix[idx] =embedding_matrix[0]
                continue
        print("embedding ended")
        return embedding_matrix,EMBEDDING_DIM        

def dataset_splitting(dataset_id,input_tensor,y):
    X_train=None
    X_test=None
    y_train=None
    y_test = None
    end = 5600
    if dataset_id!=1:
        end = 4700
    if dataset_id==3:
        end = 11240
    print(len(input_tensor),print(len(y)))
    X_train = input_tensor[0:end]
    X_test = input_tensor[end:]
    y_train = y[0:end].to_list()
    y_test = y[end:].to_list()
    encoder=LabelEncoder()
    y_train=encoder.fit_transform(y_train)
    y_train=pd.get_dummies(y_train).values
    y_test=encoder.fit_transform(y_test)
    y_test=pd.get_dummies(y_test).values
    print(encoder.classes_)
    return X_train,y_train, X_test,y_test

def dataset_splitting_over(dataset_id,input_tensor,y):
    X_train=None
    X_test=None
    y_train=None
    y_test = None
    end = 5600
    if dataset_id!=1:
        end = 4700
    if dataset_id==3:
        end = 11240
    print(len(input_tensor),print(len(y)))
    X_train = input_tensor[0:end]
    X_test = input_tensor[end:]
    y_train = y[0:end].to_list()
    y_test = y[end:].to_list()
    from imblearn.over_sampling import RandomOverSampler
    from imblearn.under_sampling import RandomUnderSampler
    oversampler = RandomOverSampler(random_state=42)
    X_train_over, y_train_over = oversampler.fit_resample(X_train, y_train)
    X_train = X_train_over
    y_train = y_train_over
    encoder=LabelEncoder()
    y_train=encoder.fit_transform(y_train)
    y_train=pd.get_dummies(y_train).values
    y_test=encoder.fit_transform(y_test)
    y_test=pd.get_dummies(y_test).values
    print(encoder.classes_)
    return X_train,y_train, X_test,y_test

def train_entry(model,output_path,X_train,y_train,X_test,y_test,name,batch_size=64,epochs= 10):
    print(len(X_test), len(y_test))
    checkpoint = ModelCheckpoint(f'/kaggle/working/{name}_best.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='max')
    history=model.fit(X_train,y_train,batch_size=batch_size,epochs= epochs,validation_data=(X_test,y_test),callbacks=[checkpoint])
    return model

def base(embedding_matrix,EMBEDDING_DIM):
    inp = Input(shape=(MAX_SEQ_LENGTH,))
    x = Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, weights=[embedding_matrix])(inp)
    x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
    x = LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1)(x)
    x= Conv1D(50,kernel_size=1,activation="relu")(x)
    x=Conv1D(250,kernel_size=1,activation="relu")(x)
    # att = SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,name='Attention',kernel_regularizer=keras.regularizers.l2(1e-4),
    #                         bias_regularizer=keras.regularizers.l1(1e-4),
    #                         attention_regularizer_weight=1e-4,attention_activation='relu')(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(250, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(6, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
def Attention_model(embedding_matrix,EMBEDDING_DIM):
    inp = Input(shape=(MAX_SEQ_LENGTH,))
    x = Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, weights=[embedding_matrix])(inp)
    x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
    x = LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1)(x)
    x=Conv1D(50,kernel_size=1,activation="relu")(x)
    x=Conv1D(250,kernel_size=1,activation="relu")(x)
    att = SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,name='Attention',kernel_regularizer=keras.regularizers.l2(1e-4),
                           bias_regularizer=keras.regularizers.l1(1e-4),
                           attention_regularizer_weight=1e-4,attention_activation='relu')(x)
    x = GlobalMaxPool1D()(att)
    x = Dense(250, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(6, activation="softmax")(x)
    #from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
    checkpoint = ModelCheckpoint('/kaggle/working/m1-{epoch:03d}-{acc:03f}-{val_accuracy:03f}.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='max')
    model1 = Model(inputs=inp, outputs=x)
    model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model1.summary()
    return model1

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.metrics import confusion_matrix,accuracy_score

def evaluation(model_name,X_test,y_test,name):
    print(model_name)
    model1 = keras.models.load_model(model_name)
    y_pred=model1.predict(X_test)
    y_test_class=np.argmax(y_test,axis=1)
    y_pred_class=np.argmax(y_pred,axis=1)
    print(y_test_class)
    cm=confusion_matrix(y_test_class,y_pred_class)
    accuracy=accuracy_score(y_test_class,y_pred_class)
    print(classification_report(y_test_class, y_pred_class, digits=5))
    print(accuracy)
    print(cm)
    total = []
    y_true=['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise']
    for i in range(0,6):
        total = 0 
        for j in range(0,6):
            total = total + cm[i][j]
        print(f"classwise-class accuracy{y_true[i]}: {cm[i][i]/total}")

    data = cm
    df_cm = pd.DataFrame(data, columns=np.unique(y_true), index = np.unique(y_true))
    sn.set(font_scale=2)#for label size
    sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 24}, fmt="d")
    plt.savefig(f'{name}_Cm.pdf', format='pdf', dpi=300)
    plt.savefig(f'{name}_Cm.png', format='png', dpi=300)
    

BiLSTM-CNN, Embedding-fastText, Datset- BEMoC

In [ ]:
dataset_id =1
stem = True
stopwords = True
embedding="fasttext"
model_name = "base"
output_path="./"
batch_size = 64
epochs =20
name="fastTextbase"
corpus,y = Dataset_prepare(dataset_id)
texts = preprocess(corpus,stem,stopwords)
input_tensor,word_index = convertToTensor(texts)
embedding_matrix, EMBEDDING_DIM = embedding_matrix_gen(embedding="fasttext",word_index=word_index)
X_train,y_train, X_test,y_test = dataset_splitting(dataset_id,input_tensor,y)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

model = None
if model_name=="base":
    model = base(embedding_matrix,EMBEDDING_DIM)
history = train_entry(model,output_path,X_train,y_train, X_val,y_val, name, batch_size=batch_size,epochs=epochs)
model.save("/kaggle/working/final.h5")
evaluation(f"/kaggle/working/{name}_best.h5", X_test,y_test,name)  

    

BiLSTM-CNN, Embedding-word2vec, Datset- BEMoC

In [ ]:
dataset_id =1
stem = True
stopwords = True
embedding="word2vec"
model_name = "base"
output_path="./"
batch_size = 64
epochs =20
name="word2vec-base"
corpus,y = Dataset_prepare(dataset_id)
texts = preprocess(corpus,stem,stopwords)
input_tensor,word_index = convertToTensor(texts)
embedding_matrix, EMBEDDING_DIM = embedding_matrix_gen(embedding="fasttext",word_index=word_index)
X_train,y_train, X_test,y_test = dataset_splitting(dataset_id,input_tensor,y)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

model = None
if model_name=="base":
    model = base(embedding_matrix,EMBEDDING_DIM)
history = train_entry(model,output_path,X_train,y_train, X_val,y_val,name, batch_size=batch_size,epochs=epochs)
model.save("/kaggle/working/final.h5")

    

BiLSTM-CNN-Attention, Embedding-fastText, Datset- BEMoC

In [ ]:
dataset_id =1
stem = True
stopwords = True
embedding="fasttext"
model_name = "attention"
output_path="./"
batch_size = 64
epochs =20
name="attention-fastText"
corpus,y = Dataset_prepare(dataset_id)
texts = preprocess(corpus,stem,stopwords)
input_tensor,word_index = convertToTensor(texts)
embedding_matrix, EMBEDDING_DIM = embedding_matrix_gen(embedding="fasttext",word_index=word_index)
X_train,y_train, X_test,y_test = dataset_splitting(dataset_id,input_tensor,y)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
model = None
if model_name=="attention":
    model = Attention_model(embedding_matrix,EMBEDDING_DIM)
history = train_entry(model,output_path,X_train,y_train, X_val,y_val,name, batch_size=batch_size,epochs=epochs)
model.save("/kaggle/working/final.h5")
y_pred=history.predict(X_test)
y_test_class=np.argmax(y_test,axis=1)
y_pred_class=np.argmax(y_pred,axis=1)
print(y_test_class)
cm=confusion_matrix(y_test_class,y_pred_class)
accuracy=accuracy_score(y_test_class,y_pred_class)
print(classification_report(y_test_class, y_pred_class, digits=5))
print(accuracy)
print(cm)
total = []
y_true=['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise']
for i in range(0,6):
    total = 0 
    for j in range(0,6):
        total = total + cm[i][j]
    print(f"classwise-class accuracy{y_true[i]}: {cm[i][i]/total}")

y_true=['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise']
data = cm
df_cm = pd.DataFrame(data, columns=np.unique(y_true), index = np.unique(y_true))
sn.set(font_scale=2)#for label size
sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 24}, fmt="d")
plt.savefig(f'{name}_Cm.pdf', format='pdf', dpi=300)
plt.savefig(f'{name}_Cm.png', format='png', dpi=300)
    

BiLSTM-CNN-Attention, Embedding-fastText, Datset- BEMoC  Oversampling

In [ ]:
dataset_id =1
stem = True
stopwords = True
embedding="fasttext"
model_name = "attention"
output_path="./"
batch_size = 64
epochs =20
name="attention-fastText-over"
corpus,y = Dataset_prepare(dataset_id)
texts = preprocess(corpus,stem,stopwords)
input_tensor,word_index = convertToTensor(texts)
embedding_matrix, EMBEDDING_DIM = embedding_matrix_gen(embedding="fasttext",word_index=word_index)
X_train,y_train, X_test,y_test = dataset_splitting_over(dataset_id,input_tensor,y)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
model = None
if model_name=="attention":
    model = Attention_model(embedding_matrix,EMBEDDING_DIM)
history = train_entry(model,output_path,X_train,y_train, X_val,y_val,name, batch_size=batch_size,epochs=epochs)
model.save("/kaggle/working/final.h5")
evaluation(f"/kaggle/working/{name}_best.h5", X_test,y_test,name) 
y_pred=history.predict(X_test)
y_test_class=np.argmax(y_test,axis=1)
y_pred_class=np.argmax(y_pred,axis=1)
print(y_test_class)
cm=confusion_matrix(y_test_class,y_pred_class)
accuracy=accuracy_score(y_test_class,y_pred_class)
print(classification_report(y_test_class, y_pred_class, digits=5))
print(accuracy)
print(cm)
total = []
y_true=['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise']
for i in range(0,6):
    total = 0 
    for j in range(0,6):
        total = total + cm[i][j]
    print(f"classwise-class accuracy{y_true[i]}: {cm[i][i]/total}")

y_true=['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise']
data = cm
df_cm = pd.DataFrame(data, columns=np.unique(y_true), index = np.unique(y_true))
sn.set(font_scale=2)#for label size
sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 24}, fmt="d")
plt.savefig(f'{name}_Cm.pdf', format='pdf', dpi=300)
plt.savefig(f'{name}_Cm.png', format='png', dpi=300) 

BiLSTM-CNN-Attention, Embedding-fastText, Datset- BanglaEmotion

In [ ]:
dataset_id =2
stem = True
stopwords = True
embedding="fasttext"
model_name = "attention"
output_path="./"
batch_size = 64
epochs =20
name="attention-fastText-banglaemotion"
corpus,y = Dataset_prepare(dataset_id)
texts = preprocess(corpus,stem,stopwords)
print(texts[3])
input_tensor,word_index = convertToTensor(texts)
print(input_tensor)
embedding_matrix, EMBEDDING_DIM = embedding_matrix_gen(embedding="fasttext",word_index=word_index)
X_train,y_train, X_test,y_test = dataset_splitting(dataset_id,input_tensor,y)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
model = None
if model_name=="attention":
    model = Attention_model(embedding_matrix,EMBEDDING_DIM)
history = train_entry(model,output_path,X_train,y_train, X_val,y_val,name, batch_size=batch_size,epochs=20)
history.save("/kaggle/working/final.h5")
y_pred=history.predict(X_test)
y_test_class=np.argmax(y_test,axis=1)
y_pred_class=np.argmax(y_pred,axis=1)
print(y_test_class)
cm=confusion_matrix(y_test_class,y_pred_class)
accuracy=accuracy_score(y_test_class,y_pred_class)
print(classification_report(y_test_class, y_pred_class, digits=5))
print(accuracy)
print(cm)
total = []
y_true=['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise']
for i in range(0,6):
    total = 0 
    for j in range(0,6):
        total = total + cm[i][j]
    print(f"classwise-class accuracy{y_true[i]}: {cm[i][i]/total}")

data = cm
df_cm = pd.DataFrame(data, columns=np.unique(y_true), index = np.unique(y_true))
sn.set(font_scale=2)#for label size
sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 24}, fmt="d")
plt.savefig(f'{name}_Cm.pdf', format='pdf', dpi=300)
plt.savefig(f'{name}_Cm.png', format='png', dpi=300)

BiLSTM-CNN-Attention, Embedding-fastText, Datset- Merged

In [ ]:
dataset_id =3
stem = True
stopwords = True
embedding="fasttext"
model_name = "attention"
output_path="./"
batch_size = 64
epochs =20
name="attention-fastText-merged"
corpus,y = Dataset_prepare(dataset_id)
texts = preprocess(corpus,stem,stopwords)
print(texts[3])
input_tensor,word_index = convertToTensor(texts)
print(input_tensor)
embedding_matrix, EMBEDDING_DIM = embedding_matrix_gen(embedding="fasttext",word_index=word_index)
X_train,y_train, X_test,y_test = dataset_splitting(dataset_id,input_tensor,y)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
model = None
if model_name=="attention":
    model = Attention_model(embedding_matrix,EMBEDDING_DIM)
history = train_entry(model,output_path,X_train,y_train, X_val,y_val,name, batch_size=batch_size,epochs=20)
history.save("/kaggle/working/final.h5")
y_pred=history.predict(X_test)
y_test_class=np.argmax(y_test,axis=1)
y_pred_class=np.argmax(y_pred,axis=1)
print(y_test_class)
cm=confusion_matrix(y_test_class,y_pred_class)
accuracy=accuracy_score(y_test_class,y_pred_class)
print(classification_report(y_test_class, y_pred_class, digits=5))
print(accuracy)
print(cm)
total = []
y_true=['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise']
for i in range(0,6):
    total = 0 
    for j in range(0,6):
        total = total + cm[i][j]
    print(f"classwise-class accuracy{y_true[i]}: {cm[i][i]/total}")

data = cm
df_cm = pd.DataFrame(data, columns=np.unique(y_true), index = np.unique(y_true))
sn.set(font_scale=2)#for label size
sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 24}, fmt="d")
plt.savefig(f'{name}_Cm.pdf', format='pdf', dpi=300)
plt.savefig(f'{name}_Cm.png', format='png', dpi=300)